In [1]:
import numpy as np

*compute_loss* computes the error of the parameters w regarding the data x and the labels y. mse = True computes the MSE error and mse = False computes the MAE error.

In [2]:
def compute_loss(y, tx, w, mse = True):
    N = y.shape[0]
    if mse:
        e = y - tx @ w
        loss = 1/(2 * N) * e.T @ e
    else:
        loss = np.mean(np.abs(y - tx @ w))
    return loss

*mse_gradient* computes the gradient of the MSE loss function regarding the labels y, the data tx and the parameters w

In [2]:
def mse_gradient(y, tx, w):
    e = y - tx @ w
    grad = -(1/y.shape[0]) * tx.T @ e
    return grad

*mae_gradient* computes the gradient of the MAE loss function regarding the labels y, the data tx and the parameters w

In [ ]:
def mae_gradient(y, tx, w):
    e = y - tx @ w
    e = np.where(e < 0, 1, -1)
    e = np.vstack((e,e)).T * tx
    return np.mean(e, axis = 0)

### least_squares_GD
*least_squares_GD* applies the full gradient descent with respect to the MSE loss function. y are the labels, tx the data, initial_w the initial vector, max_iters are the number of steps of the algorithm and gamma the step size.
at each step, this performs $\underline{w} = \underline{w} - \gamma * \nabla L(\underline{w})$.

In [2]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    w = initial_w
    for n_iter in range(max_iters):
        grad = mse_grad(y, tx, w)
        w = w - gamma * grad
    loss = compute_loss(y, tx, w)
    return loss, w

### least_squares_SGD
*least_squares_GD* applies batch gradient descent with respect to the MSE loss function. y are the labels, tx the data, initial_w the initial vector, max_iters are the number of steps of the algorithm and gamma the step size.
at each step, this performs $\underline{w}^{(t+1)} = \underline{w}^{(t)} - \gamma * \underline{g}$ where $\underline{g} = \frac{1}{|B|} \sum\limits_{n \in B} \nabla L_n(\underline{w}^{(t)})$

In [1]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma, batch_size = 1):
    
    w = initial_w
    rand_list = np.arange(y.shape[0])
    
    for n in np.arange(max_iters):
        np.random.shuffle(rand_list)
        # randomizing y and tx so we can take the first *batch_size* elements
        y = y[rand_list]
        tx = tx[rand_list]
        
        # compute loss and gradent descent
        grad = compute_stoch_gradient(y[:batch_size], tx[:batch_size,:], w)
        
        w = w - gamma * grad
        
    loss = compute_loss(y, tx, w)
    return losse, w

### least square
given tx (N, D+1) and y (N), finds the solution in terms of the lest squares, namely $arg min_{b} || Y - XB ||^2$
which is $B = (X^T X)^{-1} (X^T Y)$

In [ ]:
def least_square_loss(tx, y, w):
    e = y - tx @ w.T
    return (1/y.shape[0]) * e @ e.T

In [3]:
def least_squares(tx, y):
    XT_X = tx.T @ tx
    XT_Y = tx.T @ y
    w = np.linalg.inv(XT_X) @ XT_Y
    loss = least_square_loss(tx, y, w)
    return w, loss

### ridge regression
given tx (N, D+1), lamdba and y (N), finds the solution in terms of the lest squares, namely $arg min_{b} || Y - XB ||^2 + \lambda ||B||^2$
which is $B = (X^T X + \lambda I_{D+1})^{-1} (X^T Y)$

In [4]:
def ridge_regression_loss(tx, y, w, lambda_):
    return least_square_loss(tx, y, w) + lambda_ * w.T @ w

In [5]:
def ridge_regression(tx, y, lambda_):
    XT_X = tx.T @ tx
    XT_Y = tx.T @ y
    w = np.linalg.inv(XT_X + np.eye(tx.shape[1]) * lambda_) @ XT_Y
    loss = ridge_regression_loss(tx, y, w, lambda_)
    return w, loss

In [13]:
import numpy as np
import pandas as pd

data = pd.read_csv("train.csv")

In [6]:
def make_features(X):
    # converting -999. to nan to use np.nanmean and np.nanstd
    X = np.where(X == -999., np.nan, X)
    # standardizing the data Xd = (X_d - E[X_d])/(std(X_d))
    X, means, stds = standardize(X)
    # since data is standirdized, the mean is more or less 0 for each feature so replacing by zero is reasonable and helps computations
    X = np.where(np.isnan(X), 0, X)
    # adding the 1 padding
    return np.column_stack((np.ones(X.shape[0]), X))

In [16]:
data.head()

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,100000,s,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,...,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497
1,100001,b,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,...,-1.916,164.546,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226
2,100002,b,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,...,-2.186,260.414,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251
3,100003,b,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,...,0.060,86.062,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000
4,100004,b,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,...,-0.871,53.131,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000


In [18]:
data['Prediction'] = np.where(data['Prediction'] == 'b', 1, 0)

In [31]:
data = data.replace(-999.000, np.nan)

In [37]:
N = data['Id'].shape[0]
data.isnull().sum(axis = 0) / N

Id                             0.000000
Prediction                     0.000000
DER_mass_MMC                   0.152456
DER_mass_transverse_met_lep    0.000000
DER_mass_vis                   0.000000
DER_pt_h                       0.000000
DER_deltaeta_jet_jet           0.709828
DER_mass_jet_jet               0.709828
DER_prodeta_jet_jet            0.709828
DER_deltar_tau_lep             0.000000
DER_pt_tot                     0.000000
DER_sum_pt                     0.000000
DER_pt_ratio_lep_tau           0.000000
DER_met_phi_centrality         0.000000
DER_lep_eta_centrality         0.709828
PRI_tau_pt                     0.000000
PRI_tau_eta                    0.000000
PRI_tau_phi                    0.000000
PRI_lep_pt                     0.000000
PRI_lep_eta                    0.000000
PRI_lep_phi                    0.000000
PRI_met                        0.000000
PRI_met_phi                    0.000000
PRI_met_sumet                  0.000000
PRI_jet_num                    0.000000


In [50]:
temp = data.fillna(1)
a = np.where(temp['DER_mass_MMC'] == 1, temp['DER_mass_MMC'], -1)

In [51]:
b = data['Prediction']

In [52]:
a @ b / N

-0.3751